# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

In [14]:
EPHYS_SAMPLE_RATE = 20000

Explanation of each input and where it comes from.

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_3_alone_comp"

In [16]:
# TRIAL_LABELS_DF = pd.read_excel("/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext/results/2024_06_26_sleap_clustering/data/rce_pilot_3_alone_comp_per_video_trial_labels.xlsx")
TRIAL_LABELS_DF = pd.read_excel("../../data/sleap_id_corrected/rce_pilot_3_alone_comp_per_video_trial_labels.xlsx")

In [17]:
LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_04_spectral_and_sleap.pkl".format(OUTPUT_PREFIX))

In [18]:
FULL_LFP_TRACES_PKL = "{}_10_per_trial_spectral_bans_sleap.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [19]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [20]:
LFP_SPECTRAL_DF.columns

Index(['video_name', 'current_subject', 'frame_index', 'tone_start_frame',
       'tone_stop_frame', 'condition ', 'competition_closeness', 'notes',
       '10s_before_tone_frame', '10s_after_tone_frame',
       ...
       'vHPC_lfp_RMS_filtered', 'vHPC_lfp_mask', 'vHPC_lfp_modified_zscore',
       'vHPC_lfp_trace', 'vHPC_lfp_trace_filtered',
       'vHPC_mPFC_granger_all_frequencies_all_windows', 'vHPC_theta_band',
       'vHPC_theta_phase', 'video_timestamps', 'cluster_timestamp'],
      dtype='object', length=226)

In [21]:
TRIAL_LABELS_DF.columns

Index(['Unnamed: 0', 'session_dir', 'tone_start_frame', 'reward_start',
       'reward_dispensed', 'tone_stop_frame', 'condition ',
       'competition_closeness', 'get_reward_frame', 'out_reward_frame',
       'notes', 'experiment', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'all_subjects',
       'first_timestamp', 'last_timestamp', 'tone_start_timestamp',
       'tone_stop_timestamp', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps'],
      dtype='object')

## Merging the labels and LFP spectral

In [22]:
TRIAL_LABELS_DF.head()

,Unnamed: 0,session_dir,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,...,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,0.0,80,100.0,74.0,NaN,NaN,NaN,NaN,...,[ 0 74],[ 0 74],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,0.0,75474.0,[ 0 75474],[ 0 75474]
1,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,1272.0,1352,1372.0,1471.0,NaN,NaN,NaN,NaN,...,[1282 1283] [1285 1297] [1326 1344] [1362 1509],[1326 1328] [1331 1347] [1380 1592],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,1275889.0,1475891.0,[1285888 1286688] [1287488 1300689] [1330286 1...,[1329489 1331487] [1334489 1350089] [1383490 1...
2,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,3268.0,3348,3368.0,3469.0,NaN,NaN,NaN,NaN,...,[3281 3324] [3324 3327] [3344 3347] [3347 3523],[3327 3704],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,3275911.0,3475914.0,[3287914 3330314] [3331114 3333712] [3351715 3...,[3333515 3712519]
3,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,4466.0,4546,4566.0,4667.0,NaN,NaN,NaN,NaN,...,[4441 4476] [4476 4489] [4508 4509] [4511 4534...,[4445 4473] [4494 4551] [4566 4830],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,4475927.0,4675931.0,[4450528 4485129] [4485929 4497529] [4516929 4...,[4454926 4483129] [4503531 4560730] [4575330 4...
4,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,5465.0,5545,5565.0,5664.0,NaN,NaN,NaN,NaN,...,[5483 5512] [5552 5698],[5455 5486] [5581 5794],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,5475942.0,5675942.0,[5493742 5523542] [5563742 5709147],[5466941 5496741] [5592343 5806546]


In [23]:
LFP_SPECTRAL_DF.head()

,video_name,current_subject,frame_index,tone_start_frame,tone_stop_frame,condition,competition_closeness,notes,10s_before_tone_frame,10s_after_tone_frame,...,vHPC_lfp_RMS_filtered,vHPC_lfp_mask,vHPC_lfp_modified_zscore,vHPC_lfp_trace,vHPC_lfp_trace_filtered,vHPC_mPFC_granger_all_frequencies_all_windows,vHPC_theta_band,vHPC_theta_phase,video_timestamps,cluster_timestamp
0,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,"[34042, 34043, 34044, 34045, 34046, 34047, 340...","[34242, 34242, 34242, 34242, 34242, 34242, 342...","[34442, 34442, 34442, 34442, 34442, 34442, 344...","[4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, ...","[After trial, After trial, After trial, After ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34042, 34042, 34042, 34042, 34042, 34042, 340...","[34642, 34642, 34642, 34642, 34642, 34642, 346...",...,"[-0.079, -0.661, -0.941, -0.939, -1.488, -1.78...","[False, False, False, False, False, False, Fal...","[-0.066892564, -0.65406066, -0.93649584, -0.93...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[0.6103468, 0.5730442, 0.5342873, 0.4941693, 0...","[-0.41403934, 0.30454272, 0.38569367, 0.659321...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084..."
1,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.3,"[34042, 34043, 34044, 34045, 34046, 34047, 340...","[34242, 34242, 34242, 34242, 34242, 34242, 342...","[34442, 34442, 34442, 34442, 34442, 34442, 344...","[4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, ...","[After trial, After trial, After trial, After ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34042, 34042, 34042, 34042, 34042, 34042, 340...","[34642, 34642, 34642, 34642, 34642, 34642, 346...",...,"[-0.007, -0.037, -0.495, -0.804, -0.191, 0.058...","[False, False, False, False, False, False, Fal...","[-0.0018840783, -0.03202933, -0.4936285, -0.80...","[-0.78, -3.8999999, -51.675, -83.85, -19.89, 6...","[-0.78, -3.8999999, -51.675, -83.85, -19.89, 6...","[[0.10801135, 0.9345647, 0.71951747, 0.1227541...","[-0.19497989, -0.21380065, -0.23236576, -0.250...","[2.0032697, 2.1665337, 2.1993783, 2.2978027, 2...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084..."
2,20240322_120625_alone_comp_subj_3-3_and_3-4.1,3.3,"[34069, 34070, 34071, 34072, 34073, 34074, 340...","[34269, 34269, 34269, 34269, 34269, 34269, 342...","[34468, 34468, 34468, 34468, 34468, 34468, 344...","[3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, ...","[Subj 1 then Subj 2, Subj 1 then Subj 2, Subj ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34069, 34069, 34069, 34069, 34069, 34069, 340...","[34668, 34668, 34668, 34668, 34668, 34668, 346...",...,"[-0.413, 0.247, 0.407, -0.413, -0.615, -0.218,...","[False, False, False, False, False, False, Fal...","[-0.44384193, 0.26560622, 0.43685234, -0.44384...","[-24.765, 14.82, 24.375, -24.765, -36.855, -13...","[-24.765, 14.82, 24.375, -24.765, -36.855, -13...","[[0.014051933, 0.6209912, 0.20073363, 0.871971...","[0.069354706, 0.071877405, 0.07377358, 0.07502...","[-0.8432274, -0.42268786, -0.36932397, -0.1285...","[1384, 1983, 2770, 4156, 5541, 5541, 6927, 831...","[34103476, 34104862, 34106248, 34106248, 34107..."
3,20240322_120625_alone_comp_subj_3-3_and_3-4.1,3.4,"[34069, 34070, 34071, 34072, 34073, 34074, 340...","[34269, 34269, 34269, 34269, 34269, 34269, 342...","[34468, 34468, 34468, 34468, 34468, 34468, 344...","[3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, ...","[Subj 1 then Subj 2, Subj 1 then Subj 2, Subj ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34069, 34069, 34069, 34069, 34069, 34069, 340...","[34668, 34668, 34668, 34668, 34668, 34668, 346...",...,"[-0.469, -0.878, 0.017, 0.835, -0.533, -0.996,...","[False, False, False, False, False, False, Fal...","[-0.46027198, -0.8696384, 0.027573898, 0.84842...","[-43.289997, -80.924995

In [24]:
LFP_SPECTRAL_DF["video_name"] = LFP_SPECTRAL_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

In [25]:
LFP_SPECTRAL_DF["current_subject"] = LFP_SPECTRAL_DF["current_subject"].astype(str)


In [26]:
TRIAL_LABELS_DF["video_name"].unique()

array(['20240320_142408_alone_comp_subj_3-1_and_3-3.1.videoTimeStamps.cameraHWSync',
       '20240320_142408_alone_comp_subj_3-1_and_3-3.2.videoTimeStamps.cameraHWSync',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.1.videoTimeStamps.cameraHWSync',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.2.videoTimeStamps.cameraHWSync',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1.videoTimeStamps.cameraHWSync',
       nan,
       '20240322_120625_alone_comp_subj_3-3_and_3-4.2.videoTimeStamps.cameraHWSync',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1.videoTimeStamps.cameraHWSync',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.2.videoTimeStamps.cameraHWSync',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.1.videoTimeStamps.cameraHWSync',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.2.videoTimeStamps.cameraHWSync',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1.videoTimeStamps.cameraHWSync',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.2

In [27]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["video_name"])

In [28]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["condition "])

In [29]:
TRIAL_LABELS_DF["video_name"] = TRIAL_LABELS_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))


In [30]:
TRIAL_LABELS_DF["video_name"].unique()

array(['20240320_142408_alone_comp_subj_3-1_and_3-3.1',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.1',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.1',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.1'], dtype=object)

In [31]:
LFP_SPECTRAL_DF["video_name"].unique()

array(['20240320_171038_alone_comp_subj_4-2_and_4-3.1',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.1',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.1'], dtype=object)

In [32]:
TRIALS_AND_SPECTRAL_DF = pd.merge(TRIAL_LABELS_DF, LFP_SPECTRAL_DF, on=["video_name"], how="inner", suffixes=('', '_y'))

In [33]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF if "Unnamed" in col])

In [34]:
TRIALS_AND_SPECTRAL_DF.head()

,session_dir,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,...,vHPC_lfp_RMS_filtered,vHPC_lfp_mask,vHPC_lfp_modified_zscore,vHPC_lfp_trace,vHPC_lfp_trace_filtered,vHPC_mPFC_granger_all_frequencies_all_windows,vHPC_theta_band,vHPC_theta_phase,video_timestamps,cluster_timestamp
0,20240320_171038_alone_comp_subj_4-2_and_4-3,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,NaN,...,"[-0.079, -0.661, -0.941, -0.939, -1.488, -1.78...","[False, False, False, False, False, False, Fal...","[-0.066892564, -0.65406066, -0.93649584, -0.93...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[0.6103468, 0.5730442, 0.5342873, 0.4941693, 0...","[-0.41403934, 0.30454272, 0.38569367, 0.659321...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084..."
1,20240320_171038_alone_comp_subj_4-2_and_4-3,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,NaN,...,"[-0.007, -0.037, -0.495, -0.804, -0.191, 0.058...","[False, False, False, False, False, False, Fal...","[-0.0018840783, -0.03202933, -0.4936285, -0.80...","[-0.78, -3.8999999, -51.675, -83.85, -19.89, 6...","[-0.78, -3.8999999, -51.675, -83.85, -19.89, 6...","[[0.10801135, 0.9345647, 0.71951747, 0.1227541...","[-0.19497989, -0.21380065, -0.23236576, -0.250...","[2.0032697, 2.1665337, 2.1993783, 2.2978027, 2...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084..."
2,20240320_171038_alone_comp_subj_4-2_and_4-3,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,NaN,...,"[-0.079, -0.661, -0.941, -0.939, -1.488, -1.78...","[False, False, False, False, False, False, Fal...","[-0.066892564, -0.65406066, -0.93649584, -0.93...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[0.6103468, 0.5730442, 0.5342873, 0.4941693, 0...","[-0.41403934, 0.30454272, 0.38569367, 0.659321...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084..."
3,20240320_171038_alone_comp_subj_4-2_and_4-3,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,NaN,...,"[-0.007, -0.037, -0.495, -0.804, -0.191, 0.058...","[False, False, False, False, False, False, Fal...","[-0.0018840783, -0.03202933, -0.4936285, -0.80...","[-0.78, -3.8999999, -51.675, -83.85, -19.89, 6...","[-0.78, -3.8999999, -51.675, -83.85, -19.89, 6...","[[0.10801135, 0.9345647, 0.71951747, 0.1227541...","[-0.19497989, -0.21380065, -0.23236576, -0.250...","[2.0032697, 2.1665337, 2.1993783, 2.2978027, 2...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084..."
4,20240320_171038_alone_comp_subj_4-2_and_4-3,36339.0,36419,36439.0,36538.0,4.3,Subj 2 blocking Subj 1,NaN,NaN,NaN,...,"[-0.079, -0.661, -0.941, -0.939, -1.488, -1.78...","[False, False, False, False, False, False, Fal...","[-0.066892564, -0.65406066, -0.93649584, -0.93...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[0.6103468, 0.5730442, 0.5342873, 0.4941693, 0...","[-0.41403934, 0.30454272, 0.38569367, 0.659321...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084..."


In [35]:
TRIALS_AND_SPECTRAL_DF.tail()

,session_dir,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,...,vHPC_lfp_RMS_filtered,vHPC_lfp_mask,vHPC_lfp_modified_zscore,vHPC_lfp_trace,vHPC_lfp_trace_filtered,vHPC_mPFC_granger_all_frequencies_all_windows,vHPC_theta_band,vHPC_theta_phase,video_timestamps,cluster_timestamp
243,20240323_165815_alone_comp_subj_4-2_and_4-4,57880.0,57960,57980.0,58079.0,4.4,Subj 2 then Subj 1,NaN,NaN,NaN,...,"[0.285, 0.161, -0.38, -0.921, -0.873, -0.731, ...","[False, False, False, False, False, False, Fal...","[0.3113077, 0.1764077, -0.41507694, -1.0065615...","[5.85, 3.3149998, -7.7999997, -18.914999, -17....","[5.85, 3.3149998, -7.7999997, -18.914999, -17....","[[0.22426029, 0.031661306, 0.23187897, 0.50685...","[-0.17116006, -0.19133626, -0.21119694, -0.230...","[1.9537605, 2.088994, 2.1246827, 2.2100766, 2....","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[36182281, 36183667, 36183667, 36185052, 36186..."
244,20240323_165815_alone_comp_subj_4-2_and_4-4,60077.0,60157,60177.0,60276.0,4.4,Subj 2 blocking Subj 1,NaN,NaN,4.2 just jumping around on the right side of port,...,"[0.083, 0.316, 0.588, 0.714, 0.662, 0.246, -0....","[False, False, False, False, False, False, Fal...","[0.0997087, 0.32845217, 0.59629714, 0.7194666,...","[8.19, 31.005, 57.719997, 70.005, 64.935, 24.1...","[8.19, 31.005, 57.719997, 70.005, 64.935, 24.1...","[[3.099082, 0.46414718, 0.33549076, 0.6002563,...","[-0.2993394, -0.27316827, -0.24553603, -0.2164...","[3.108997, -2.4551952, -2.344967, -2.1353333, ...","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[36182281, 36183667, 36183667, 36185052, 36186..."
245,20240323_165815_alone_comp_subj_4-2_and_4-4,60077.0,60157,60177.0,60276.0,4.4,Subj 2 blocking Subj 1,NaN,NaN,4.2 just jumping around on the right side of port,...,"[0.285, 0.161, -0.38, -0.921, -0.873, -0.731, ...","[False, False, False, False, False, False, Fal...","[0.3113077, 0.1764077, -0.41507694, -1.0065615...","[5.85, 3.3149998, -7.7999997, -18.914999, -17....","[5.85, 3.3149998, -7.7999997, -18.914999, -17....","[[0.22426029, 0.031661306, 0.23187897, 0.50685...","[-0.17116006, -0.19133626, -0.21119694, -0.230...","[1.9537605, 2.088994, 2.1246827, 2.2100766, 2....","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[36182281, 36183667, 36183667, 36185052, 36186..."
246,20240323_165815_alone_comp_subj_4-2_and_4-4,61174.0,61254,61274.0,61375.0,tie,Close Call,NaN,NaN,NaN,...,"[0.083, 0.316, 0.588, 0.714, 0.662, 0.246, -0....","[False, False, False, False, False, False, Fal...","[0.0997087, 0.32845217, 0.59629714, 0.7194666,...","[8.19, 31.005, 57.719997, 70.005, 64.935, 24.1...","[8.19, 31.005, 57.719997, 70.005, 64.935, 24.1...","[[3.099082, 0.46414718, 0.33549076, 0.6002563,...","[-0.2993394, -0.27316827, -0.24553603, -0.2164...","[3.108997, -2.4551952, -2.344967, -2.1353333, ...","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[36182281, 36183667, 36183667, 36185052, 36186..."
247,20240323_165815_alone_comp_subj_4-2_and_4-4,61174.0,61254,61274.0,61375.0,tie,Close Call,NaN,NaN,NaN,...,"[0.285, 0.161, -0.38, -0.921, -0.873, -0.731, ...","[False, False, False, False, False, False, Fal...","[0.3113077, 0.1764077, -0.41507694, -1.0065615...","[5.85, 3.3149998, -7.7999997, -18.914999, -17....","[5.85, 3.3149998, -7.7999997, -18.914999, -17....","[[0.22426029, 0.031661306, 0.23187897, 0.50685...","[-0.17116006, -0.19133626, -0.21119694, -0.230...","[1.9537605, 2.088994, 2.1246827, 2.2100766, 2....","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[36182281, 36183667, 36183667, 36185052, 36186..."


In [36]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF.columns if "_y" in col], errors="ignore")

In [37]:
timestamps_col = ["first_timestamp", "last_timestamp", "tone_start_timestamp", "tone_stop_timestamp"]

In [38]:
for col in timestamps_col:
    TRIALS_AND_SPECTRAL_DF[col] = TRIALS_AND_SPECTRAL_DF[col].astype(int)

# Calculating the relevant timestamps

In [39]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] - 10 * EPHYS_SAMPLE_RATE

In [40]:
TRIALS_AND_SPECTRAL_DF["post_trial_end_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"] + 10 * EPHYS_SAMPLE_RATE

In [41]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"].head()

0    34080576
1    34080576
2    35080586
3    35080586
4    36180600
Name: baseline_start_timestamp, dtype: int64

In [42]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    34280576
1    34280576
2    35280586
3    35280586
4    36380600
Name: tone_start_timestamp, dtype: int64

In [43]:
TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"].head()

0    34480576
1    34480576
2    35480585
3    35480585
4    36580602
Name: tone_stop_timestamp, dtype: int64

In [44]:
TRIALS_AND_SPECTRAL_DF.head()

,session_dir,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,...,vHPC_lfp_modified_zscore,vHPC_lfp_trace,vHPC_lfp_trace_filtered,vHPC_mPFC_granger_all_frequencies_all_windows,vHPC_theta_band,vHPC_theta_phase,video_timestamps,cluster_timestamp,baseline_start_timestamp,post_trial_end_timestamp
0,20240320_171038_alone_comp_subj_4-2_and_4-3,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,NaN,...,"[-0.066892564, -0.65406066, -0.93649584, -0.93...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[0.6103468, 0.5730442, 0.5342873, 0.4941693, 0...","[-0.41403934, 0.30454272, 0.38569367, 0.659321...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...",34080576,34680576
1,20240320_171038_alone_comp_subj_4-2_and_4-3,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,NaN,...,"[-0.0018840783, -0.03202933, -0.4936285, -0.80...","[-0.78, -3.8999999, -51.675, -83.85, -19.89, 6...","[-0.78, -3.8999999, -51.675, -83.85, -19.89, 6...","[[0.10801135, 0.9345647, 0.71951747, 0.1227541...","[-0.19497989, -0.21380065, -0.23236576, -0.250...","[2.0032697, 2.1665337, 2.1993783, 2.2978027, 2...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...",34080576,34680576
2,20240320_171038_alone_comp_subj_4-2_and_4-3,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,NaN,...,"[-0.066892564, -0.65406066, -0.93649584, -0.93...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[0.6103468, 0.5730442, 0.5342873, 0.4941693, 0...","[-0.41403934, 0.30454272, 0.38569367, 0.659321...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...",35080586,35680585
3,20240320_171038_alone_comp_subj_4-2_and_4-3,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,NaN,...,"[-0.0018840783, -0.03202933, -0.4936285, -0.80...","[-0.78, -3.8999999, -51.675, -83.85, -19.89, 6...","[-0.78, -3.8999999, -51.675, -83.85, -19.89, 6...","[[0.10801135, 0.9345647, 0.71951747, 0.1227541...","[-0.19497989, -0.21380065, -0.23236576, -0.250...","[2.0032697, 2.1665337, 2.1993783, 2.2978027, 2...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...",35080586,35680585
4,20240320_171038_alone_comp_subj_4-2_and_4-3,36339.0,36419,36439.0,36538.0,4.3,Subj 2 blocking Subj 1,NaN,NaN,NaN,...,"[-0.066892564, -0.65406066, -0.93649584, -0.93...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[0.6103468, 0.5730442, 0.5342873, 0.4941693, 0...","[-0.41403934, 0.30454272, 0.38569367, 0.659321...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...",36180600,36780602


# Filtering out the SLEAP posed for during trials

In [45]:
list(TRIALS_AND_SPECTRAL_DF.columns)

['session_dir',
 'tone_start_frame',
 'reward_start',
 'reward_dispensed',
 'tone_stop_frame',
 'condition ',
 'competition_closeness',
 'get_reward_frame',
 'out_reward_frame',
 'notes',
 'experiment',
 'box_1_port_entry_frames',
 'box_2_port_entry_frames',
 'video_name',
 'all_subjects',
 'first_timestamp',
 'last_timestamp',
 'tone_start_timestamp',
 'tone_stop_timestamp',
 'box_1_port_entry_timestamps',
 'box_2_port_entry_timestamps',
 'current_subject',
 'frame_index',
 '10s_before_tone_frame',
 '10s_after_tone_frame',
 'subject_thorax_to_agent_thorax',
 'nose_to_reward_port_sum',
 'nose_to_reward_port_diff',
 'thorax_velocity_sum',
 'thorax_velocity_diff',
 'to_reward_port_angle_sum',
 'to_reward_port_angle_diff',
 'subject_nose_to_reward_port',
 'subject_thorax_velocity',
 'subject_to_reward_port_angle',
 'agent_nose_to_reward_port',
 'agent_thorax_velocity',
 'agent_to_reward_port_angle',
 'closebool_subject_nose_to_reward_port',
 'closebool_agent_nose_to_reward_port',
 'moving

In [46]:
TRIALS_AND_SPECTRAL_DF["video_frame"] = TRIALS_AND_SPECTRAL_DF["video_timestamps"].apply(lambda x: np.array(list(range(len(x)))) + 1)

In [47]:
TRIALS_AND_SPECTRAL_DF["video_frame"].head().apply(lambda x: x.shape)

0    (64255,)
1    (64255,)
2    (64255,)
3    (64255,)
4    (64255,)
Name: video_frame, dtype: object

In [49]:
['session_dir',
 'tone_start_frame',
 'reward_start',
 'reward_dispensed',
 'tone_stop_frame',
 'condition ',
 'competition_closeness',
 'get_reward_frame',
 'out_reward_frame',
 'notes',
 'experiment',
 'box_1_port_entry_frames',
 'box_2_port_entry_frames',
 'video_name',
 'all_subjects',
 'first_timestamp',
 'last_timestamp',
 'tone_start_timestamp',
 'tone_stop_timestamp',
 'box_1_port_entry_timestamps',
 'box_2_port_entry_timestamps',
 'current_subject',
 'frame_index',
 '10s_before_tone_frame',
 '10s_after_tone_frame',
 'subject_thorax_to_agent_thorax',
 'nose_to_reward_port_sum',
 'nose_to_reward_port_diff',
 'thorax_velocity_sum',
 'thorax_velocity_diff',
 'to_reward_port_angle_sum',
 'to_reward_port_angle_diff',
 'subject_nose_to_reward_port',
 'subject_thorax_velocity',
 'subject_to_reward_port_angle',
 'agent_nose_to_reward_port',
 'agent_thorax_velocity',
 'agent_to_reward_port_angle',
 'closebool_subject_nose_to_reward_port',
 'closebool_agent_nose_to_reward_port',
 'movingbool_subject_thorax_velocity',
 'movingbool_agent_thorax_velocity',
 'manual_cluster_id',
 'standard_embedding_x',
 'kmeans_cluster',
 'subject_locations',
 'agent_locations',
 'subject_thorax',
 'subject_nose',
 'subject_tail_base',
 'agent_thorax',
 'agent_nose',
 'agent_tail_base',
 'sleap_name',
 'video_id',
 'agent']

['session_dir',
 'tone_start_frame',
 'reward_start',
 'reward_dispensed',
 'tone_stop_frame',
 'condition ',
 'competition_closeness',
 'get_reward_frame',
 'out_reward_frame',
 'notes',
 'experiment',
 'box_1_port_entry_frames',
 'box_2_port_entry_frames',
 'video_name',
 'all_subjects',
 'first_timestamp',
 'last_timestamp',
 'tone_start_timestamp',
 'tone_stop_timestamp',
 'box_1_port_entry_timestamps',
 'box_2_port_entry_timestamps',
 'current_subject',
 'frame_index',
 '10s_before_tone_frame',
 '10s_after_tone_frame',
 'subject_thorax_to_agent_thorax',
 'nose_to_reward_port_sum',
 'nose_to_reward_port_diff',
 'thorax_velocity_sum',
 'thorax_velocity_diff',
 'to_reward_port_angle_sum',
 'to_reward_port_angle_diff',
 'subject_nose_to_reward_port',
 'subject_thorax_velocity',
 'subject_to_reward_port_angle',
 'agent_nose_to_reward_port',
 'agent_thorax_velocity',
 'agent_to_reward_port_angle',
 'closebool_subject_nose_to_reward_port',
 'closebool_agent_nose_to_reward_port',
 'moving

In [50]:
# sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "locations" in col or "velocity" in col or "to_reward_port" in col or "video_frame" in col]

In [51]:
sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "thorax" in col or "nose" in col or "reward_port" in col or "standard_embedding" in col or "cluster" in col or "frame_index" in col]


In [52]:
sleap_columns

['frame_index',
 'subject_thorax_to_agent_thorax',
 'nose_to_reward_port_sum',
 'nose_to_reward_port_diff',
 'thorax_velocity_sum',
 'thorax_velocity_diff',
 'to_reward_port_angle_sum',
 'to_reward_port_angle_diff',
 'subject_nose_to_reward_port',
 'subject_thorax_velocity',
 'subject_to_reward_port_angle',
 'agent_nose_to_reward_port',
 'agent_thorax_velocity',
 'agent_to_reward_port_angle',
 'closebool_subject_nose_to_reward_port',
 'closebool_agent_nose_to_reward_port',
 'movingbool_subject_thorax_velocity',
 'movingbool_agent_thorax_velocity',
 'manual_cluster_id',
 'standard_embedding_x',
 'kmeans_cluster',
 'subject_thorax',
 'subject_nose',
 'agent_thorax',
 'agent_nose',
 'cluster_timestamp']

In [53]:
for col in sleap_columns:
    print(col)
    print(TRIALS_AND_SPECTRAL_DF[col].apply(lambda x: x.shape).iloc[0])

frame_index
(12593,)
subject_thorax_to_agent_thorax
(12593,)
nose_to_reward_port_sum
(12593,)
nose_to_reward_port_diff
(12593,)
thorax_velocity_sum
(12593,)
thorax_velocity_diff
(12593,)
to_reward_port_angle_sum
(12593,)
to_reward_port_angle_diff
(12593,)
subject_nose_to_reward_port
(12593,)
subject_thorax_velocity
(12593,)
subject_to_reward_port_angle
(12593,)
agent_nose_to_reward_port
(12593,)
agent_thorax_velocity
(12593,)
agent_to_reward_port_angle
(12593,)
closebool_subject_nose_to_reward_port
(12593,)
closebool_agent_nose_to_reward_port
(12593,)
movingbool_subject_thorax_velocity
(12593,)
movingbool_agent_thorax_velocity
(12593,)
manual_cluster_id
(12593,)
standard_embedding_x
(12593,)
kmeans_cluster
(12593,)
subject_thorax
(12593, 2)
subject_nose
(12593, 2)
agent_thorax
(12593, 2)
agent_nose
(12593, 2)
cluster_timestamp
(12593,)


In [54]:
for col in sorted(sleap_columns):
    updated_item_col = "pre_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "pre_and_post_video_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["cluster_timestamp"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["cluster_timestamp"], items=x[col])[0], axis=1)

pre_and_post_agent_nose
pre_and_post_agent_nose_to_reward_port
pre_and_post_agent_thorax
pre_and_post_agent_thorax_velocity
pre_and_post_agent_to_reward_port_angle
pre_and_post_closebool_agent_nose_to_reward_port
pre_and_post_closebool_subject_nose_to_reward_port
pre_and_post_cluster_timestamp
pre_and_post_frame_index
pre_and_post_kmeans_cluster
pre_and_post_manual_cluster_id
pre_and_post_movingbool_agent_thorax_velocity
pre_and_post_movingbool_subject_thorax_velocity
pre_and_post_nose_to_reward_port_diff
pre_and_post_nose_to_reward_port_sum
pre_and_post_standard_embedding_x
pre_and_post_subject_nose
pre_and_post_subject_nose_to_reward_port
pre_and_post_subject_thorax
pre_and_post_subject_thorax_to_agent_thorax
pre_and_post_subject_thorax_velocity
pre_and_post_subject_to_reward_port_angle
pre_and_post_thorax_velocity_diff
pre_and_post_thorax_velocity_sum
pre_and_post_to_reward_port_angle_diff
pre_and_post_to_reward_port_angle_sum


In [55]:
TRIALS_AND_SPECTRAL_DF.head()

,session_dir,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,...,pre_and_post_subject_nose_to_reward_port,pre_and_post_subject_thorax,pre_and_post_subject_thorax_to_agent_thorax,pre_and_post_subject_thorax_velocity,pre_and_post_subject_to_reward_port_angle,pre_and_post_thorax_velocity_diff,pre_and_post_thorax_velocity_sum,pre_and_post_to_reward_port_angle_diff,pre_and_post_to_reward_port_angle_sum,pre_and_post_video_timestamps
0,20240320_171038_alone_comp_subj_4-2_and_4-3,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,NaN,...,"[2.5980964, 3.2073143, 3.9916637, 4.8566847, 5...","[[54.093177424666216, 30.357495891388663], [53...","[4.9368286, 4.95431, 4.9935803, 5.1178784, 5.3...","[5.1050973, 5.082959, 5.072279, 5.091807, 5.14...","[2.4313493, 2.3651402, 2.2818353, 2.199549, 2....","[5.6331806, 5.7147374, 5.6083183, 5.3814564, 5...","[15.843375, 16.048643, 16.248707, 16.423662, 1...","[0.6321054, 0.6899271, 0.74678916, 0.802343, 0...","[4.8800473, 4.662217, 4.401958, 4.117365, 3.82...","[34081302, 34082688, 34084074, 34084074, 34085..."
1,20240320_171038_alone_comp_subj_4-2_and_4-3,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,NaN,...,"[1.1949695, 0.93336076, 0.738537, 0.5684192, 0...","[[52.330529932218774, 33.8102838308305], [52.8...","[4.9368286, 4.95431, 4.9935803, 5.1178784, 5.3...","[10.738277, 10.965685, 11.176429, 11.331856, 1...","[2.4486978, 2.297077, 2.1201227, 1.9178162, 1....","[5.6331806, 5.7147374, 5.6083183, 5.3814564, 5...","[15.843375, 16.048643, 16.248707, 16.423662, 1...","[0.6321054, 0.6899271, 0.74678916, 0.802343, 0...","[4.8800473, 4.662217, 4.401958, 4.117365, 3.82...","[34081302, 34082688, 34084074, 34084074, 34085..."
2,20240320_171038_alone_comp_subj_4-2_and_4-3,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,NaN,...,"[1.088969, 1.0757009, 1.0131229, 0.9762328, 0....","[[59.86917104209446, 31.176008001006366], [59....","[24.583914, 23.842857, 23.053654, 22.21829, 21...","[0.84502804, 0.7296608, 0.62726414, 0.54466546...","[0.17591879, 0.17943281, 0.17603831, 0.2038250...","[15.349337, 18.314651, 21.41159, 24.472757, 27...","[17.039392, 19.773973, 22.66612, 25.562088, 28...","[1.9538506, 2.020497, 2.0900402, 2.0998404, 2....","[2.3056881, 2.3793628, 2.442117, 2.5074904, 2....","[35081900, 35081900, 35083286, 35084672, 35085..."
3,20240320_171038_alone_comp_subj_4-2_and_4-3,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,NaN,...,"[25.323456, 23.973078, 22.660395, 21.498892, 2...","[[47.29296806743566, 9.135700788934573], [46.8...","[24.583914, 23.842857, 23.053654, 22.21829, 21...","[16.194365, 19.044312, 22.038855, 25.017424, 2...","[2.1297693, 2.19993, 2.2660787, 2.3036654, 2.3...","[15.349337, 18.314651, 21.41159, 24.472757, 27...","[17.039392, 19.773973, 22.66612, 25.562088, 28...","[1.9538506, 2.020497, 2.0900402, 2.0998404, 2....","[2.3056881, 2.3793628, 2.442117, 2.5074904, 2....","[35081900, 35081900, 35083286, 35084672, 35085..."
4,20240320_171038_alone_comp_subj_4-2_and_4-3,36339.0,36419,36439.0,36538.0,4.3,Subj 2 blocking Subj 1,NaN,NaN,NaN,...,"[0.7897185, 0.7472973, 0.7605625, 0.8177425, 0...","[[60.38185337098157, 31.177069002360277], [60....","[13.881315, 14.176304, 14.438505, 14.65004, 14...","[0.49317342, 0.4948187, 0.509421, 0.54049957, ...","[0.5789979, 0.5902529, 0.5984625, 0.60776705, ...","[5.934906, 6.0408955, 5.9854646, 5.8209095, 5....","[6.936893, 7.039064, 7.0063467, 6.898076, 6.69...","[0.3554064, 0.39349312, 0.42495528, 0.46008515...","[0.8359046, 0.79514295, 0.75987095, 0.7393834,...","[36180895, 36182281, 36183666, 36183666, 36186..."


In [56]:
TRIALS_AND_SPECTRAL_DF["pre_and_post_frame_index"].iloc[0].shape

(599,)

In [57]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=sleap_columns + ["video_timestamps"], errors="ignore")

In [58]:
for col in TRIALS_AND_SPECTRAL_DF.columns:
    print(col)

session_dir
tone_start_frame
reward_start
reward_dispensed
tone_stop_frame
condition 
competition_closeness
get_reward_frame
out_reward_frame
notes
experiment
box_1_port_entry_frames
box_2_port_entry_frames
video_name
all_subjects
first_timestamp
last_timestamp
tone_start_timestamp
tone_stop_timestamp
box_1_port_entry_timestamps
box_2_port_entry_timestamps
current_subject
10s_before_tone_frame
10s_after_tone_frame
subject_locations
agent_locations
subject_tail_base
agent_tail_base
sleap_name
video_id
agent
BLA-to-LH_granger_gamma
BLA-to-LH_granger_theta
BLA-to-MD_granger_gamma
BLA-to-MD_granger_theta
BLA-to-mPFC_granger_gamma
BLA-to-mPFC_granger_theta
BLA-to-vHPC_granger_gamma
BLA-to-vHPC_granger_theta
BLA_LH_coherence_all_frequencies_all_windows
BLA_LH_coherence_gamma
BLA_LH_coherence_theta
BLA_LH_granger_all_frequencies_all_windows
BLA_MD_coherence_all_frequencies_all_windows
BLA_MD_coherence_gamma
BLA_MD_coherence_theta
BLA_MD_granger_all_frequencies_all_windows
BLA_RMS_filtered_pow

- Filtering coherence

In [61]:
coherence_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "coherence" in col and "timestamps" not in col and "calculation" not in col]

In [62]:
coherence_columns

['BLA_LH_coherence_all_frequencies_all_windows',
 'BLA_LH_coherence_gamma',
 'BLA_LH_coherence_theta',
 'BLA_MD_coherence_all_frequencies_all_windows',
 'BLA_MD_coherence_gamma',
 'BLA_MD_coherence_theta',
 'BLA_mPFC_coherence_all_frequencies_all_windows',
 'BLA_mPFC_coherence_gamma',
 'BLA_mPFC_coherence_theta',
 'BLA_vHPC_coherence_all_frequencies_all_windows',
 'BLA_vHPC_coherence_gamma',
 'BLA_vHPC_coherence_theta',
 'LH_MD_coherence_all_frequencies_all_windows',
 'LH_MD_coherence_gamma',
 'LH_MD_coherence_theta',
 'LH_mPFC_coherence_all_frequencies_all_windows',
 'LH_mPFC_coherence_gamma',
 'LH_mPFC_coherence_theta',
 'LH_vHPC_coherence_all_frequencies_all_windows',
 'LH_vHPC_coherence_gamma',
 'LH_vHPC_coherence_theta',
 'MD_mPFC_coherence_all_frequencies_all_windows',
 'MD_mPFC_coherence_gamma',
 'MD_mPFC_coherence_theta',
 'MD_vHPC_coherence_all_frequencies_all_windows',
 'MD_vHPC_coherence_gamma',
 'MD_vHPC_coherence_theta',
 'mPFC_vHPC_coherence_all_frequencies_all_windows',


In [63]:
for col in sorted(coherence_columns):
    updated_item_col = "pre_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "pre_and_post_video_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)

pre_and_post_BLA_LH_coherence_all_frequencies_all_windows
pre_and_post_BLA_LH_coherence_gamma
pre_and_post_BLA_LH_coherence_theta
pre_and_post_BLA_MD_coherence_all_frequencies_all_windows
pre_and_post_BLA_MD_coherence_gamma
pre_and_post_BLA_MD_coherence_theta
pre_and_post_BLA_mPFC_coherence_all_frequencies_all_windows
pre_and_post_BLA_mPFC_coherence_gamma
pre_and_post_BLA_mPFC_coherence_theta
pre_and_post_BLA_vHPC_coherence_all_frequencies_all_windows
pre_and_post_BLA_vHPC_coherence_gamma
pre_and_post_BLA_vHPC_coherence_theta
pre_and_post_LH_MD_coherence_all_frequencies_all_windows
pre_and_post_LH_MD_coherence_gamma
pre_and_post_LH_MD_coherence_theta
pre_and_post_LH_mPFC_coherence_all_frequencies_all_windows
pre_and_post_LH_mPFC_coherence_gamma
pre_and_post_LH_mPFC_coherence_theta
pre_and_post_LH_vHPC_coherence_all_frequencies_all_windows
pre_and_post_LH_vHPC_coherence_gamma
pre_and_post_LH_vHPC_coherence_theta
pre_and_post_MD_mPFC_coherence_all_frequencies_all_windows
pre_and_post_MD_

In [64]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=coherence_columns + ["coherence_timestamps"], errors="ignore")

- Filtering Grangers

In [65]:
granger_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "granger" in col and "timestamps" not in col and "calculation" not in col]

In [66]:
granger_columns

['BLA-to-LH_granger_gamma',
 'BLA-to-LH_granger_theta',
 'BLA-to-MD_granger_gamma',
 'BLA-to-MD_granger_theta',
 'BLA-to-mPFC_granger_gamma',
 'BLA-to-mPFC_granger_theta',
 'BLA-to-vHPC_granger_gamma',
 'BLA-to-vHPC_granger_theta',
 'BLA_LH_granger_all_frequencies_all_windows',
 'BLA_MD_granger_all_frequencies_all_windows',
 'BLA_mPFC_granger_all_frequencies_all_windows',
 'BLA_vHPC_granger_all_frequencies_all_windows',
 'LH-to-BLA_granger_gamma',
 'LH-to-BLA_granger_theta',
 'LH-to-MD_granger_gamma',
 'LH-to-MD_granger_theta',
 'LH-to-mPFC_granger_gamma',
 'LH-to-mPFC_granger_theta',
 'LH-to-vHPC_granger_gamma',
 'LH-to-vHPC_granger_theta',
 'LH_BLA_granger_all_frequencies_all_windows',
 'LH_MD_granger_all_frequencies_all_windows',
 'LH_mPFC_granger_all_frequencies_all_windows',
 'LH_vHPC_granger_all_frequencies_all_windows',
 'MD-to-BLA_granger_gamma',
 'MD-to-BLA_granger_theta',
 'MD-to-LH_granger_gamma',
 'MD-to-LH_granger_theta',
 'MD-to-mPFC_granger_gamma',
 'MD-to-mPFC_granger_t

In [67]:
for col in sorted(granger_columns):
    updated_item_col = "pre_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "pre_and_post_video_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)

pre_and_post_BLA-to-LH_granger_gamma
pre_and_post_BLA-to-LH_granger_theta
pre_and_post_BLA-to-MD_granger_gamma
pre_and_post_BLA-to-MD_granger_theta
pre_and_post_BLA-to-mPFC_granger_gamma
pre_and_post_BLA-to-mPFC_granger_theta
pre_and_post_BLA-to-vHPC_granger_gamma
pre_and_post_BLA-to-vHPC_granger_theta
pre_and_post_BLA_LH_granger_all_frequencies_all_windows
pre_and_post_BLA_MD_granger_all_frequencies_all_windows
pre_and_post_BLA_mPFC_granger_all_frequencies_all_windows
pre_and_post_BLA_vHPC_granger_all_frequencies_all_windows
pre_and_post_LH-to-BLA_granger_gamma
pre_and_post_LH-to-BLA_granger_theta
pre_and_post_LH-to-MD_granger_gamma
pre_and_post_LH-to-MD_granger_theta
pre_and_post_LH-to-mPFC_granger_gamma
pre_and_post_LH-to-mPFC_granger_theta
pre_and_post_LH-to-vHPC_granger_gamma
pre_and_post_LH-to-vHPC_granger_theta
pre_and_post_LH_BLA_granger_all_frequencies_all_windows
pre_and_post_LH_MD_granger_all_frequencies_all_windows
pre_and_post_LH_mPFC_granger_all_frequencies_all_windows
pr

In [68]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=granger_columns + ["granger_timestamps"], errors="ignore")

In [69]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

- Filtering power

In [70]:
TRIALS_AND_SPECTRAL_DF.head()

,session_dir,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,...,pre_and_post_vHPC-to-LH_granger_gamma,pre_and_post_vHPC-to-LH_granger_theta,pre_and_post_vHPC-to-MD_granger_gamma,pre_and_post_vHPC-to-MD_granger_theta,pre_and_post_vHPC-to-mPFC_granger_gamma,pre_and_post_vHPC-to-mPFC_granger_theta,pre_and_post_vHPC_BLA_granger_all_frequencies_all_windows,pre_and_post_vHPC_LH_granger_all_frequencies_all_windows,pre_and_post_vHPC_MD_granger_all_frequencies_all_windows,pre_and_post_vHPC_mPFC_granger_all_frequencies_all_windows
0,20240320_171038_alone_comp_subj_4-2_and_4-3,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,NaN,...,"[0.29341307, 0.25543022, 0.4201393, 0.23980951...","[0.36782616, 0.11848986, 0.26258618, 0.3678115...","[0.23130372, 0.34455413, 0.32136595, 0.1711519...","[0.3305366, 0.1900366, 0.5451826, 0.35900232, ...","[0.18996, 0.2313756, 0.45686305, 0.1539646, 0....","[0.24734256, 0.11689982, 0.43787938, 0.2189229...","[[0.7969552, 0.896725, 0.08472422, 0.22447705,...","[[0.11179353, 0.2347677, 0.5620465, 0.19969913...","[[1.2762377, 1.2222601, 0.035040043, 0.5407100...","[[0.01619947, 0.124842755, 0.4138201, 0.149170..."
1,20240320_171038_alone_comp_subj_4-2_and_4-3,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,NaN,...,"[0.5062664, 0.6013324, 0.28935644, 0.28739107,...","[0.36696273, 0.6597051, 0.19064282, 0.39824545...","[0.58782995, 0.46289966, 0.17136228, 0.1721866...","[0.23794852, 0.19862981, 0.11563269, 0.3981060...","[0.38799196, 0.29099765, 0.1223909, 0.21023442...","[0.4256485, 0.17132095, 0.3653476, 0.7137943, ...","[[0.01593203, 0.50195557, 0.22257225, 1.051627...","[[0.030171767, 0.51494586, 0.05360611, 0.49936...","[[1.6665556, 0.20019405, 0.11537983, 0.0011932...","[[0.0009881377, 0.9771604, 0.574515, 0.2478132..."
2,20240320_171038_alone_comp_subj_4-2_and_4-3,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,NaN,...,"[0.23565084, 0.29949525, 0.40756822, 0.3720342...","[0.29373866, 0.19913526, 0.1664011, 0.17306124...","[0.29494444, 0.33364385, 0.43299717, 0.3429591...","[0.3036561, 0.25521982, 0.21036552, 0.2728724,...","[0.19576988, 0.21245046, 0.26954073, 0.1542914...","[0.26033425, 0.35530025, 0.075279415, 0.164272...","[[0.0015281317, 0.05445077, 0.4672157, 0.14554...","[[0.2036718, 0.031650987, 0.44773144, 0.328611...","[[0.66735643, 0.22344528, 0.23367827, 0.264800...","[[3.2664258, 1.0008569, 0.2694132, 0.08409576,..."
3,20240320_171038_alone_comp_subj_4-2_and_4-3,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,NaN,...,"[0.069989085, 0.23998056, 0.29364488, 0.182408...","[0.19689235, 0.12293133, 0.18649417, 0.255562,...","[0.21834221, 0.20445994, 0.22933356, 0.2893294...","[0.24399506, 0.21722521, 0.40527433, 0.4494708...","[nan, nan, 0.27557155, 0.18831533, 0.22206363,...","[nan, nan, 0.19225794, 0.17318204, 0.4158725, ...","[[1.3576767, 0.18696061, 0.53456825, 0.6180225...","[[0.021285018, 0.2659307, 0.62544596, 0.100141...","[[2.6762187, 0.359536, 0.591506, 0.23778938, 0...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
4,20240320_171038_alone_comp_subj_4-2_and_4-3,36339.0,36419,36439.0,36538.0,4.3,Subj 2 blocking Subj 1,NaN,NaN,NaN,...,"[0.3760613, 0.28269434, 0.31337386, 0.23031126...","[0.09998439, 0.92171395, 1.0438794, 0.19538805...","[0.31572416, 0.1987081, 0.18472315, 0.15923928...","[0.23193137, 0.7758891, 0.39863747, 0.43584633...","[0.43207794, 0.22308315, 0.27651992, 0.0629055...","[0.29729372, 0.37963903, 0.27517694, 0.3632434...","[[0.74546915, 0.22318698, 0.009117729, 0.07482...","[[1.2072806, 0.22529954, 0.022428008, 0.122314...","[[0.9502404, 0.2883093, 0.040734246, 0.3167789...","[[1.5025154, 0.13349208, 0.04087703, 0.0351425..."


In [71]:
power_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "power" in col and "timestamps" not in col and "calculation" not in col]

In [72]:
power_columns

['BLA_RMS_filtered_power_all_frequencies_all_windows',
 'BLA_RMS_filtered_power_gamma',
 'BLA_RMS_filtered_power_theta',
 'LH_RMS_filtered_power_all_frequencies_all_windows',
 'LH_RMS_filtered_power_gamma',
 'LH_RMS_filtered_power_theta',
 'MD_RMS_filtered_power_all_frequencies_all_windows',
 'MD_RMS_filtered_power_gamma',
 'MD_RMS_filtered_power_theta',
 'mPFC_RMS_filtered_power_all_frequencies_all_windows',
 'mPFC_RMS_filtered_power_gamma',
 'mPFC_RMS_filtered_power_theta',
 'vHPC_RMS_filtered_power_all_frequencies_all_windows',
 'vHPC_RMS_filtered_power_gamma',
 'vHPC_RMS_filtered_power_theta']

In [73]:
for col in sorted(power_columns):
    updated_item_col = "pre_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "pre_and_post_video_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)


pre_and_post_BLA_RMS_filtered_power_all_frequencies_all_windows
pre_and_post_BLA_RMS_filtered_power_gamma
pre_and_post_BLA_RMS_filtered_power_theta
pre_and_post_LH_RMS_filtered_power_all_frequencies_all_windows
pre_and_post_LH_RMS_filtered_power_gamma
pre_and_post_LH_RMS_filtered_power_theta
pre_and_post_MD_RMS_filtered_power_all_frequencies_all_windows
pre_and_post_MD_RMS_filtered_power_gamma
pre_and_post_MD_RMS_filtered_power_theta
pre_and_post_mPFC_RMS_filtered_power_all_frequencies_all_windows
pre_and_post_mPFC_RMS_filtered_power_gamma
pre_and_post_mPFC_RMS_filtered_power_theta
pre_and_post_vHPC_RMS_filtered_power_all_frequencies_all_windows
pre_and_post_vHPC_RMS_filtered_power_gamma
pre_and_post_vHPC_RMS_filtered_power_theta


In [75]:
TRIALS_AND_SPECTRAL_DF["pre_and_post_power_timestamps"]

KeyError: 'pre_and_post_power_timestamps'

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=power_columns + ["power_timestamps"], errors="ignore")

In [ ]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

# Filtering out phase

In [ ]:
lfp_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "trace" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
band_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "band" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
phase_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "phase" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
phase_columns = phase_columns + band_columns +lfp_columns

In [ ]:
phase_columns

In [ ]:
for col in phase_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_lfp_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[0], axis=1)

In [ ]:
for col in phase_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_lfp_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_lfp_timestamps"]

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=phase_columns + ["lfp_timestamps"], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

# Filtering out spikes

In [ ]:
raise ValueError()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"]).astype(int), axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"]).astype(int), axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"].iloc[0]

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1], axis=1)
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1].T, axis=1)
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=["spike_clusters", "spike_times", "neuron_average_fr", "neuron_average_timestamps",], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"].iloc[0].shape

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"].iloc[0].shape

# Filtering for rows that are in the video

In [ ]:
TRIALS_AND_SPECTRAL_DF["in_video"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: x["start_frame"] <= x["trial_video_frame"][0] <= x["stop_frame"], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["in_video"]].reset_index()

In [ ]:
TRIALS_AND_SPECTRAL_DF

In [ ]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
raise ValueError()